In [1]:
import requests
import pandas as pd
from xml.etree import ElementTree
import xml.etree.cElementTree as ET
from collections import defaultdict
from ratelimit import *

ImportError: No module named 'ratelimit'

In [33]:
from password import password
import json

In [35]:
# url
url ='http://ieeexplore.ieee.org/gateway/ipsSearch.jsp?ti=Prisoners&hc=10'

In [36]:
@rate_limited(1)
def fetch_xml(url):
    """A function which creates the request to the API and returns the XML
    file.
    """
    response = requests.get(url, stream=True)

    if response.status_code != 200:
        raise APIError(response.status_code)

    root = ET.parse(response.raw).getroot()
    return root

In [37]:
root = fetch_xml(url)

In [38]:
def xml_to_dict(branch):
    """Branch to dictionary"""
    article = defaultdict()
    for at in branch.iter():
        article.update({at.tag: at.text})

    return article

In [83]:
def iee_to_axelbib(article):
    """A function which takes a dictionary with structure of the IEEE results
    and transform it to a structure understandable by Axelbib"""
    keys = ['key', 'title', 'abstract', 'author', 'date', 'journal', 'pages', 'labels', 'list_strategies']

    article['author'] = []
    for i in article['authors'].split(';  '):
        article['author'].append({'name': i})
    article['date'] = {'year': int(article['py'])}
    article['journal'] = article.pop('pubtitle')
    # article['notes'] = article.pop('pdf')
    
    first_name, last_name = article['author'][0]['name'].split(' ')
    year = article['date']['year']
    article['key'] = '{}{}'.format(last_name, year)
    article['pages'], article['labels'], article['list_strategies'] = "", [], []

    post = {key: article[key] for key in keys}
    
    return post

In [84]:
def post_to_axelbib(post):
    """A function for posting to Axelbib"""
    url = 'http://127.0.0.1:8000/article/'
    headers = {'Content-Type': 'application/json'}

    r = requests.post(url, data=json.dumps(post),
                      auth=('nikoleta', password), headers=headers)
    return r.status_code

In [99]:
parents = root.getchildren()

In [100]:
for _ in range(2): parents.remove(parents[0])

In [101]:
parents

[<Element 'document' at 0x7fa013b65408>,
 <Element 'document' at 0x7fa013b6d048>,
 <Element 'document' at 0x7fa013b6d8b8>,
 <Element 'document' at 0x7fa013b70408>,
 <Element 'document' at 0x7fa013b70d68>,
 <Element 'document' at 0x7fa013b727c8>,
 <Element 'document' at 0x7fa013b75228>,
 <Element 'document' at 0x7fa013b75d68>,
 <Element 'document' at 0x7fa013b76a48>,
 <Element 'document' at 0x7fa013b7a5e8>]

In [97]:
for document in parents:
    article = xml_to_dict(document)
    post = iee_to_axelbib(article)
    send = post_to_axelbib(post)
    
    print(send)

201
201
